In [ ]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
#from sklearn.feature_extraction.text import TfidfVectorizer

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import PorterStemmer
from nltk.stem.lancaster import LancasterStemmer
from sklearn.model_selection import train_test_split
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from nltk.corpus import stopwords
#from nltk.stemporter import PorterStemmer

# Import libraries for feature 
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2


from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,classification_report
from sklearn import metrics

import warnings
warnings.filterwarnings("ignore")


from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive
%cd /gdrive


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [ ]:
#Read files
textfile = r'/gdrive/My Drive/CIS508/Assignment-5/Comments.csv'
textData = pd.read_csv(textfile) #creates a dataframe

CustInfofile = r'/gdrive/My Drive/CIS508/Assignment-5/Customers.csv'
CustInfoData = pd.read_csv(CustInfofile)  #creates a dataframe

print(textData.shape)
print(CustInfoData.shape)


(2070, 2)
(2070, 17)


In [ ]:
#Extract target column from Customer Info file
y_train = CustInfoData["TARGET"]
X_train = CustInfoData.drop(columns=["TARGET"]) #extracting training data without the target column
                     
print(X_train.shape)
print(textData.shape)
textData.head()
print(X_train)
print(y_train)

(2070, 16)
(2070, 2)
        ID Sex Status  ...  Paymethod  LocalBilltype LongDistanceBilltype
0        1   F      S  ...         CC         Budget       Intnl_discount
1        6   M      M  ...         CH      FreeLocal             Standard
2        8   M      M  ...         CC      FreeLocal             Standard
3       11   M      S  ...         CC         Budget             Standard
4       14   F      M  ...         CH         Budget       Intnl_discount
...    ...  ..    ...  ...        ...            ...                  ...
2065  3821   F      S  ...         CC      FreeLocal             Standard
2066  3822   F      S  ...       Auto         Budget             Standard
2067  3823   F      M  ...         CH         Budget             Standard
2068  3824   F      M  ...         CC      FreeLocal             Standard
2069  3825   F      S  ...         CC      FreeLocal             Standard

[2070 rows x 16 columns]
0       Cancelled
1         Current
2         Current
3         C

In [ ]:
#Using Snowball_Stemmer

# Use English stemmer.
stemmer1 = SnowballStemmer("english")
#Tokenize - Split the sentences to lists of words
textData['CommentsTokenized'] = textData['Comments'].apply(word_tokenize)

export_csv = textData.to_csv(r'/gdrive/My Drive/CIS508/Assignment-5/TextDataTokenized1_Snow.csv')

#Now do stemming - create a new dataframe to store stemmed version
newTextData1=pd.DataFrame()
newTextData1=textData.drop(columns=["CommentsTokenized","Comments"])
newTextData1['CommentsTokenizedStemmed'] = textData['CommentsTokenized'].apply(lambda x: [stemmer1.stem(y) for y in x]) # Stem every word.

export_csv = newTextData1.to_csv(r'/gdrive/My Drive/CIS508/Assignment-5/newTextDataTS_Snow.csv')

#Join stemmed strings
newTextData1['CommentsTokenizedStemmed'] = newTextData1['CommentsTokenizedStemmed'].apply(lambda x: " ".join(x))

export_csv = newTextData1.to_csv(r'/gdrive/My Drive/CIS508/Assignment-5/newTextData-Joined_Snow.csv')



In [ ]:
#Bag-Of-Words, Term - Document Matrix
#Learn the vocabulary dictionary and return term-document matrix.
count_vect = CountVectorizer(stop_words='english',lowercase=False)
TD_counts1 = count_vect.fit_transform(newTextData1.CommentsTokenizedStemmed)
TD_counts1.dtype
print(count_vect.get_feature_names())
print("No. of stem words in SNOWBALL stemmer is : ",len(count_vect.get_feature_names()))
DF_TD_Counts=pd.DataFrame(TD_counts1.toarray())
print(DF_TD_Counts)
export_csv = DF_TD_Counts.to_csv(r'/gdrive/My Drive/CIS508/Assignment-5/TD_counts-TokenizedStemmed.csv')


['3399', '3g', 'abysm', 'access', 'accessori', 'adapt', 'add', 'addit', 'additon', 'address', 'adit', 'adress', 'advertis', 'afraid', 'alway', 'angel', 'angri', 'ani', 'anoth', 'anyth', 'anytim', 'area', 'asap', 'ask', 'bad', 'basic', 'bateri', 'batteri', 'becaus', 'believ', 'better', 'bigger', 'book', 'bought', 'brain', 'bring', 'built', 'busi', 'button', 'buy', 'cancel', 'cancer', 'car', 'care', 'carrier', 'caus', 'cc', 'cell', 'certain', 'chang', 'charg', 'charger', 'check', 'chip', 'citi', 'claim', 'cleariti', 'cold', 'comapr', 'compani', 'compar', 'competit', 'complain', 'complaint', 'concept', 'connect', 'consisit', 'consist', 'constan', 'contact', 'continu', 'contract', 'correct', 'cost', 'coupl', 'cover', 'coverag', 'creat', 'credit', 'cstmer', 'cstmr', 'current', 'cust', 'custom', 'customr', 'date', 'day', 'dead', 'decent', 'defect', 'deo', 'did', 'die', 'differ', 'difficult', 'digiti', 'direct', 'disabl', 'doe', 'don', 'dont', 'drop', 'dure', 'easier', 'effect', 'encount', 'e

In [ ]:
#Porter Stemmer

# Use English stemmer.
stemmer2 = PorterStemmer()
#Tokenize - Split the sentences to lists of words
textData['CommentsTokenized'] = textData['Comments'].apply(word_tokenize)

export_csv = textData.to_csv(r'/gdrive/My Drive/CIS508/Assignment-5/TextDataTokenized1_porter.csv')

#Now do stemming - create a new dataframe to store stemmed version
newTextData2=pd.DataFrame()
newTextData2=textData.drop(columns=["CommentsTokenized","Comments"])
newTextData2['CommentsTokenizedStemmed'] = textData['CommentsTokenized'].apply(lambda x: [stemmer2.stem(y) for y in x]) # Stem every word.

export_csv = newTextData2.to_csv(r'/gdrive/My Drive/CIS508/Assignment-5/newTextDataTS_porter.csv')

#Join stemmed strings
newTextData2['CommentsTokenizedStemmed'] = newTextData2['CommentsTokenizedStemmed'].apply(lambda x: " ".join(x))

export_csv = newTextData2.to_csv(r'/gdrive/My Drive/CIS508/Assignment-5/newTextData-Joined_porter.csv')

In [ ]:
#Bag-Of-Words and Term - Document Matrix
#Learn the vocabulary dictionary and return term-document matrix.
count_vect = CountVectorizer(stop_words='english',lowercase=False)
TD_counts2 = count_vect.fit_transform(newTextData2.CommentsTokenizedStemmed)
TD_counts2.dtype
print(count_vect.get_feature_names())
print("No. of stem words in PORTER stemmer is : ",len(count_vect.get_feature_names()))
DF_TD_Counts=pd.DataFrame(TD_counts2.toarray())
print(DF_TD_Counts)
export_csv = DF_TD_Counts.to_csv(r'/gdrive/My Drive/CIS508/Assignment-5/TD_counts-TokenizedStemmed.csv')

['3399', '3g', 'As', 'CC', 'He', 'If', 'In', 'Is', 'It', 'We', 'abysm', 'access', 'accessori', 'adapt', 'add', 'addit', 'additon', 'address', 'adit', 'adress', 'advertis', 'afraid', 'alway', 'angel', 'angri', 'ani', 'anoth', 'anyth', 'anytim', 'area', 'asap', 'ask', 'bad', 'basic', 'bateri', 'batteri', 'becaus', 'believ', 'better', 'bigger', 'book', 'bought', 'brain', 'bring', 'built', 'busi', 'button', 'buy', 'cancel', 'cancer', 'car', 'care', 'carrier', 'caus', 'cc', 'cell', 'certain', 'chang', 'charg', 'charger', 'check', 'chip', 'citi', 'claim', 'cleariti', 'cold', 'comapr', 'compani', 'compar', 'competit', 'complain', 'complaint', 'concept', 'connect', 'consisit', 'consist', 'constanli', 'contact', 'continu', 'contract', 'correct', 'cost', 'coupl', 'cover', 'coverag', 'creat', 'credit', 'cstmer', 'cstmr', 'current', 'cust', 'custom', 'customr', 'date', 'day', 'dead', 'decent', 'defect', 'deo', 'did', 'die', 'differ', 'difficult', 'digiti', 'direct', 'disabl', 'doe', 'don', 'dont',

In [ ]:
#LANCASTER STEMMER

# Use English stemmer.
stemmer3 = LancasterStemmer()
#Tokenize - Split the sentences to lists of words
textData['CommentsTokenized'] = textData['Comments'].apply(word_tokenize)

export_csv = textData.to_csv(r'/gdrive/My Drive/CIS508/Assignment-5/TextDataTokenized1_lancaster.csv')

#Now do stemming - create a new dataframe to store stemmed version
newTextData=pd.DataFrame()
newTextData=textData.drop(columns=["CommentsTokenized","Comments"])
newTextData['CommentsTokenizedStemmed'] = textData['CommentsTokenized'].apply(lambda x: [stemmer3.stem(y) for y in x]) # Stem every word.

export_csv = newTextData.to_csv(r'/gdrive/My Drive/CIS508/Assignment-5/newTextDataTS_lancaster.csv')

#Join stemmed strings
newTextData['CommentsTokenizedStemmed'] = newTextData['CommentsTokenizedStemmed'].apply(lambda x: " ".join(x))

export_csv = newTextData.to_csv(r'/gdrive/My Drive/CIS508/Assignment-5/newTextData-Joined_lancaster.csv')

In [ ]:
#Bag-Of-Words and Term - Document Matrix
count_vect = CountVectorizer(stop_words='english',lowercase=False)
TD_counts3 = count_vect.fit_transform(newTextData.CommentsTokenizedStemmed)
TD_counts3.dtype
print(count_vect.get_feature_names())
print("No. of stem words in LANCASTER stemmer is : ",len(count_vect.get_feature_names()))
DF_TD_Counts=pd.DataFrame(TD_counts3.toarray())
print(DF_TD_Counts)
export_csv = DF_TD_Counts.to_csv(r'/gdrive/My Drive/CIS508/Assignment-5/TD_counts-TokenizedStemmed.csv')

['3399', '3g', 'abysm', 'access', 'ad', 'adapt', 'addit', 'additon', 'address', 'adit', 'adress', 'advert', 'afraid', 'aft', 'al', 'alway', 'angel', 'angry', 'anoth', 'anyth', 'anytim', 'ar', 'asap', 'ask', 'bad', 'bas', 'batery', 'battery', 'becaus', 'believ', 'bet', 'big', 'bil', 'book', 'bought', 'brain', 'bring', 'built', 'busy', 'button', 'buy', 'cal', 'cancel', 'car', 'carry', 'caus', 'cc', 'cel', 'certain', 'chang', 'charg', 'check', 'chip', 'city', 'claim', 'clear', 'cold', 'comapr', 'comp', 'company', 'competit', 'complain', 'complaint', 'conceiv', 'connect', 'consisit', 'consist', 'const', 'contact', 'continu', 'contract', 'correct', 'cost', 'coupl', 'cov', 'cre', 'credit', 'cstmer', 'cstmr', 'cur', 'cust', 'custom', 'customr', 'dat', 'day', 'dead', 'dec', 'defect', 'deo', 'did', 'die', 'diff', 'difficult', 'digit', 'direct', 'dis', 'doe', 'don', 'dont', 'drop', 'dur', 'dying', 'easy', 'effect', 'encount', 'end', 'enemy', 'equip', 'ev', 'everytim', 'everywh', 'evrey', 'exact'

In [ ]:
#Compute TF-IDF Matrix
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(TD_counts2)
print(X_train_tfidf.shape)
DF_TF_IDF=pd.DataFrame(X_train_tfidf.toarray())
print(DF_TF_IDF)


(2070, 366)
      0    1    2    3         4    5    ...  360  361  362  363  364  365
0     0.0  0.0  0.0  0.0  0.000000  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
1     0.0  0.0  0.0  0.0  0.000000  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
2     0.0  0.0  0.0  0.0  0.000000  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
3     0.0  0.0  0.0  0.0  0.000000  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
4     0.0  0.0  0.0  0.0  0.000000  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
...   ...  ...  ...  ...       ...  ...  ...  ...  ...  ...  ...  ...  ...
2065  0.0  0.0  0.0  0.0  0.000000  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
2066  0.0  0.0  0.0  0.0  0.407251  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
2067  0.0  0.0  0.0  0.0  0.000000  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
2068  0.0  0.0  0.0  0.0  0.000000  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
2069  0.0  0.0  0.0  0.0  0.407251  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0

[2070 rows x 366 columns]


In [ ]:
#Feature selection
new_DF_TF_IDF = SelectKBest(score_func=chi2, k=50).fit_transform(DF_TF_IDF,y_train)
new_DF_TF_IDF.shape

DF_TF_IDF_SelectedFeatures= pd.DataFrame(new_DF_TF_IDF)
print(DF_TF_IDF_SelectedFeatures)




       0         1    2    3    4    5   ...   44   45        46   47   48   49
0     0.0  0.000000  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.000000  0.0  0.0  0.0
1     0.0  0.000000  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.000000  0.0  0.0  0.0
2     0.0  0.000000  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.000000  0.0  0.0  0.0
3     0.0  0.000000  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.000000  0.0  0.0  0.0
4     0.0  0.000000  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.348322  0.0  0.0  0.0
...   ...       ...  ...  ...  ...  ...  ...  ...  ...       ...  ...  ...  ...
2065  0.0  0.000000  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.000000  0.0  0.0  0.0
2066  0.0  0.000000  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.000000  0.0  0.0  0.0
2067  0.0  0.000000  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.000000  0.0  0.0  0.0
2068  0.0  0.772949  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.000000  0.0  0.0  0.0
2069  0.0  0.000000  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.000000  0.0  0.0  0.0

[2070 rows x 50 columns]


In [ ]:
#Merge files

print(CustInfoData.shape)

combined=pd.concat([CustInfoData,DF_TF_IDF_SelectedFeatures], axis=1)
print(combined.shape)
print(combined)



(2070, 17)
(2070, 67)
        ID Sex Status  Children  Est_Income  ...   45        46   47   48   49
0        1   F      S         1    38000.00  ...  0.0  0.000000  0.0  0.0  0.0
1        6   M      M         2    29616.00  ...  0.0  0.000000  0.0  0.0  0.0
2        8   M      M         0    19732.80  ...  0.0  0.000000  0.0  0.0  0.0
3       11   M      S         2       96.33  ...  0.0  0.000000  0.0  0.0  0.0
4       14   F      M         2    52004.80  ...  0.0  0.348322  0.0  0.0  0.0
...    ...  ..    ...       ...         ...  ...  ...       ...  ...  ...  ...
2065  3821   F      S         0    78851.30  ...  0.0  0.000000  0.0  0.0  0.0
2066  3822   F      S         1    17540.70  ...  0.0  0.000000  0.0  0.0  0.0
2067  3823   F      M         0    83891.90  ...  0.0  0.000000  0.0  0.0  0.0
2068  3824   F      M         2    28220.80  ...  0.0  0.000000  0.0  0.0  0.0
2069  3825   F      S         0    28589.10  ...  0.0  0.000000  0.0  0.0  0.0

[2070 rows x 67 columns]


In [ ]:
#Do one Hot encoding for categorical features
categorical_features = ["Sex","Status","Car_Owner","Paymethod","LocalBilltype","LongDistanceBilltype"]
print(categorical_features)
Onehencode = pd.get_dummies(combined,columns=categorical_features)
print(Onehencode.shape)
export_csv= Onehencode.to_csv(r'/gdrive/My Drive/CIS508/Assignment-5/combined_one_hot.csv')



['Sex', 'Status', 'Car_Owner', 'Paymethod', 'LocalBilltype', 'LongDistanceBilltype']
(2070, 75)


In [ ]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(Onehencode.drop(columns=["TARGET"]), Onehencode["TARGET"], test_size=0.20, random_state=42)
print('Training dataset shape:', X_train1.shape, y_train1)
print('Testing dataset shape:', X_test1.shape, y_test1)


Training dataset shape: (1656, 74) 849       Current
1043    Cancelled
175       Current
1228      Current
538     Cancelled
          ...    
1638      Current
1095      Current
1130      Current
1294      Current
860       Current
Name: TARGET, Length: 1656, dtype: object
Testing dataset shape: (414, 74) 1181      Current
69        Current
351     Cancelled
1163    Cancelled
429       Current
          ...    
1532      Current
1671      Current
416       Current
2023      Current
1428      Current
Name: TARGET, Length: 414, dtype: object


In [ ]:
#Construct a Random Forest Classifier on text data
clf=RandomForestClassifier()
RF_text = clf.fit(X_train1,y_train1)
print("Accuracy score (training): {0:.6f}".format(clf.score(X_test1, y_test1)))
rf_predictions = clf.predict(X_test1)
print("Confusion Matrix:")
print(confusion_matrix(y_test1, rf_predictions))
print("Classification Report")
print(classification_report(y_test1, rf_predictions))

Accuracy score (training): 0.847826
Confusion Matrix:
[[124  33]
 [ 30 227]]
Classification Report
              precision    recall  f1-score   support

   Cancelled       0.81      0.79      0.80       157
     Current       0.87      0.88      0.88       257

    accuracy                           0.85       414
   macro avg       0.84      0.84      0.84       414
weighted avg       0.85      0.85      0.85       414



In [ ]:
#Construct a  Decision Tree Classifier on text data
clf=DecisionTreeClassifier()
RF_text = clf.fit(X_train1,y_train1)
print("Accuracy score (training): {0:.6f}".format(clf.score(X_test1, y_test1)))
rf_predictions = clf.predict(X_test1)
print("Confusion Matrix:")
print(confusion_matrix(y_test1, rf_predictions))
print("Classification Report")
print(classification_report(y_test1, rf_predictions))

Accuracy score (training): 0.826087
Confusion Matrix:
[[122  35]
 [ 37 220]]
Classification Report
              precision    recall  f1-score   support

   Cancelled       0.77      0.78      0.77       157
     Current       0.86      0.86      0.86       257

    accuracy                           0.83       414
   macro avg       0.82      0.82      0.82       414
weighted avg       0.83      0.83      0.83       414



In [ ]:
#Construct a GRADIENT BOOSTING Classifier on text data
clf=GradientBoostingClassifier()
RF_text = clf.fit(X_train1,y_train1)
print("Accuracy score (training): {0:.6f}".format(clf.score(X_test1, y_test1)))
rf_predictions = clf.predict(X_test1)
print("Confusion Matrix:")
print(confusion_matrix(y_test1, rf_predictions))
print("Classification Report")
print(classification_report(y_test1, rf_predictions))

Accuracy score (training): 0.840580
Confusion Matrix:
[[114  43]
 [ 23 234]]
Classification Report
              precision    recall  f1-score   support

   Cancelled       0.83      0.73      0.78       157
     Current       0.84      0.91      0.88       257

    accuracy                           0.84       414
   macro avg       0.84      0.82      0.83       414
weighted avg       0.84      0.84      0.84       414



In [ ]:
#Wraper type feature selection - Random Forest
# Build RF classifier to use in feature selection
clf = RandomForestClassifier()

# Build step forward feature selection
sfs1 = sfs(clf,
           k_features=5,
           forward=True,
           floating=False,
           verbose=2,
           scoring='accuracy',
           cv=5)

# Perform SFFS
sfs1 = sfs1.fit(DF_TF_IDF,y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 366 out of 366 | elapsed:   27.4s finished

[2019-12-14 06:17:27] Features: 1/5 -- score: 0.6159469643320449[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 365 out of 365 | elapsed:   27.7s finished

[2019-12-14 06:17:55] Features: 2/5 -- score: 0.6198105401921403[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 364 out of 364 | elapsed:   28.0s finished

[2019-12-14 06:18:23] Features: 3/5 -- score: 0.622710260630549[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 o

In [ ]:
# Which features?
feat_cols = list(sfs1.k_feature_idx_)
print(feat_cols)
SF=DF_TF_IDF.iloc[:,feat_cols]
print(SF)

[22, 205, 250, 306, 354]
      22   205  250  306  354
0     0.0  0.0  0.0  0.0  0.0
1     0.0  0.0  0.0  0.0  0.0
2     0.0  0.0  0.0  0.0  0.0
3     0.0  0.0  0.0  0.0  0.0
4     0.0  0.0  0.0  0.0  0.0
...   ...  ...  ...  ...  ...
2065  0.0  0.0  0.0  0.0  0.0
2066  0.0  0.0  0.0  0.0  0.0
2067  0.0  0.0  0.0  0.0  0.0
2068  0.0  0.0  0.0  0.0  0.0
2069  0.0  0.0  0.0  0.0  0.0

[2070 rows x 5 columns]


In [ ]:
#Merge files

print(CustInfoData.shape)

combined=pd.concat([CustInfoData,SF], axis=1)
print(combined.shape)
print(combined)


(2070, 17)
(2070, 22)
        ID Sex Status  Children  Est_Income  ...   22  205  250  306  354
0        1   F      S         1    38000.00  ...  0.0  0.0  0.0  0.0  0.0
1        6   M      M         2    29616.00  ...  0.0  0.0  0.0  0.0  0.0
2        8   M      M         0    19732.80  ...  0.0  0.0  0.0  0.0  0.0
3       11   M      S         2       96.33  ...  0.0  0.0  0.0  0.0  0.0
4       14   F      M         2    52004.80  ...  0.0  0.0  0.0  0.0  0.0
...    ...  ..    ...       ...         ...  ...  ...  ...  ...  ...  ...
2065  3821   F      S         0    78851.30  ...  0.0  0.0  0.0  0.0  0.0
2066  3822   F      S         1    17540.70  ...  0.0  0.0  0.0  0.0  0.0
2067  3823   F      M         0    83891.90  ...  0.0  0.0  0.0  0.0  0.0
2068  3824   F      M         2    28220.80  ...  0.0  0.0  0.0  0.0  0.0
2069  3825   F      S         0    28589.10  ...  0.0  0.0  0.0  0.0  0.0

[2070 rows x 22 columns]


In [ ]:
#Do one Hot encoding for categorical features
categorical_features = ["Sex","Status","Car_Owner","Paymethod","LocalBilltype","LongDistanceBilltype"]
print(categorical_features)
Onehencode = pd.get_dummies(combined,columns=categorical_features)
print(Onehencode.shape)
export_csv= Onehencode.to_csv(r'/gdrive/My Drive/CIS508/Assignment-5/Onehencode.csv')

['Sex', 'Status', 'Car_Owner', 'Paymethod', 'LocalBilltype', 'LongDistanceBilltype']
(2070, 30)


In [ ]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(Onehencode.drop(columns=["TARGET"]), Onehencode["TARGET"], test_size=0.20, random_state=42)
print('Training dataset shape:', X_train2.shape, y_train2)
print('Testing dataset shape:', X_test2.shape, y_test2)

In [ ]:
#Construct a Random Forest Classifier on text data
clf=RandomForestClassifier()
RF_text = clf.fit(X_train2,y_train2)
print("Accuracy score (training): {0:.6f}".format(clf.score(X_test2, y_test2)))
rf_predictions = clf.predict(X_test2)
print("Confusion Matrix:")
print(confusion_matrix(y_test2, rf_predictions))
print("Classification Report")
print(classification_report(y_test2, rf_predictions))

Accuracy score (training): 0.847826
Confusion Matrix:
[[126  31]
 [ 32 225]]
Classification Report
              precision    recall  f1-score   support

   Cancelled       0.80      0.80      0.80       157
     Current       0.88      0.88      0.88       257

    accuracy                           0.85       414
   macro avg       0.84      0.84      0.84       414
weighted avg       0.85      0.85      0.85       414

